In [1]:
#### Petey Dola's code

# Install NFL api package
import nfl_data_py as nfl
import requests
import numpy as np
import pandas as pd
import csv

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Define the years and columns you want to pull data for
years = [2020,2021,2022,2023]
# Pull the play-by-play data
pbp_data = nfl.import_pbp_data(years, downcast=False)

2020 done.
2021 done.
2022 done.
2023 done.


In [3]:
pbp_data.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,offense_players,defense_players,n_offense,n_defense,ngs_air_yards,time_to_throw,was_pressure,route,defense_man_zone_type,defense_coverage_type
0,1.0,2020_01_ARI_SF,2020091311,SF,ARI,REG,1,None,None,None,...,,,0.0,0.0,NaN,NaN,None,None,None,None
1,39.0,2020_01_ARI_SF,2020091311,SF,ARI,REG,1,SF,home,ARI,...,00-0033258;00-0034490;00-0033862;00-0029747;00...,00-0034073;00-0034730;00-0033783;00-0031961;00...,11.0,11.0,NaN,NaN,None,None,None,None
2,54.0,2020_01_ARI_SF,2020091311,SF,ARI,REG,1,SF,home,ARI,...,00-0029892;00-0034847;00-0033221;00-0034860;00...,00-0029585;00-0035236;00-0031557;00-0035705;00...,11.0,11.0,4.19,2.436,False,OUT,MAN_COVERAGE,COVER_1
3,93.0,2020_01_ARI_SF,2020091311,SF,ARI,REG,1,SF,home,ARI,...,00-0029892;00-0034847;00-0033221;00-0034860;00...,00-0029585;00-0035236;00-0031557;00-0035705;00...,11.0,11.0,NaN,NaN,None,None,None,None
4,118.0,2020_01_ARI_SF,2020091311,SF,ARI,REG,1,SF,home,ARI,...,00-0029892;00-0034847;00-0034860;00-0033221;00...,00-0029585;00-0035236;00-0031557;00-0029747;00...,11.0,11.0,NaN,NaN,None,None,None,None


In [4]:
pbp_data.describe()

,play_id,week,yardline_100,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,quarter_end,drive,sp,qtr,...,xyac_success,xyac_fd,xpass,pass_oe,defenders_in_box,number_of_pass_rushers,n_offense,n_defense,ngs_air_yards,time_to_throw
count,196726.000000,196726.000000,182854.000000,196713.000000,196713.000000,196713.000000,196726.000000,194502.000000,196726.000000,196726.000000,...,68951.000000,68951.000000,150124.000000,146241.000000,142635.000000,82959.000000,188761.000000,188761.000000,70951.000000,73621.000000
mean,2117.833738,9.818463,48.445388,409.762695,805.960709,1705.855480,0.017466,11.492000,0.074840,2.567119,...,0.780041,0.580860,0.627144,-0.804388,6.348947,4.240902,10.216920,10.215071,7.926246,2.774677
std,1237.474701,5.541970,24.290279,282.759932,561.295935,1059.715258,0.131000,6.652709,0.263134,1.132173,...,0.250552,0.354591,0.240130,42.329561,1.064734,0.857100,2.828841,2.828964,9.926030,1.045142
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,...,0.017248,0.007925,0.010774,-99.512959,0.000000,0.000000,0.000000,0.000000,-14.450000,0.329000
25%,1053.000000,5.000000,31.000000,144.000000,274.000000,782.000000,0.000000,6.000000,0.000000,2.000000,...,0.562875,0.242992,0.456315,-42.223412,6.000000,4.000000,11.000000,11.000000,1.580000,2.102000
50%,2103.000000,10.000000,50.000000,390.000000,789.000000,1800.000000,0.000000,11.000000,0.000000,3.000000,...,0.983562,0.482523,0.597190,4.096949,6.000000,4.000000,11.000000,11.000000,5.330000,2.603000
75%,3155.000000,15.000000,70.000000,655.000000,1289.000000,2598.000000,0.000000,17.000000,0.000000,4.000000,...,1.000000,0.999135,0.855315,34.026760,7.000000,5.000000,11.000000,11.000000,12.690000,3.204000
max,5523.000000,22.000000,99.000000,900.000000,1800.000000,3600.000000,1.000000,35.000000,1.000000,5.000000,...,1.000000,1.000000,0.997902,97.995057,11.000000,10.000000,19.000000,22.000000,65.000000,14.281000


In [5]:
pbp_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196726 entries, 0 to 196725
Columns: 390 entries, play_id to defense_coverage_type
dtypes: float64(205), int32(7), int64(1), object(177)
memory usage: 580.1+ MB


In [6]:
# checking each column
for column in pbp_data.columns:
    print(column)

play_id
game_id
old_game_id
home_team
away_team
season_type
week
posteam
posteam_type
defteam
side_of_field
yardline_100
game_date
quarter_seconds_remaining
half_seconds_remaining
game_seconds_remaining
game_half
quarter_end
drive
sp
qtr
down
goal_to_go
time
yrdln
ydstogo
ydsnet
desc
play_type
yards_gained
shotgun
no_huddle
qb_dropback
qb_kneel
qb_spike
qb_scramble
pass_length
pass_location
air_yards
yards_after_catch
run_location
run_gap
field_goal_result
kick_distance
extra_point_result
two_point_conv_result
home_timeouts_remaining
away_timeouts_remaining
timeout
timeout_team
td_team
td_player_name
td_player_id
posteam_timeouts_remaining
defteam_timeouts_remaining
total_home_score
total_away_score
posteam_score
defteam_score
score_differential
posteam_score_post
defteam_score_post
score_differential_post
no_score_prob
opp_fg_prob
opp_safety_prob
opp_td_prob
fg_prob
safety_prob
td_prob
extra_point_prob
two_point_conversion_prob
ep
epa
total_home_epa
total_away_epa
total_home_rush_epa


In [7]:
### AUSTIN CHANGED
pbp_home = pbp_data[['old_game_id','home_team','away_team', 'posteam_type','shotgun', 'total_home_score',
                     'air_yards', 'yards_gained', 'game_date']].copy()

In [8]:
pbp_home['game_date'] = pd.to_datetime(pbp_home['game_date'])

###AUSTIN CHANGED - warning was rising because of above cell not being a copy() but being a slice of the original dataframe
#pbp_home.loc[:,'game_date'] = pd.to_datetime(pbp_home.loc[:,'game_date']) # this will also get around it if you don't use copy()

pbp_home['game_date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 196726 entries, 0 to 196725
Series name: game_date
Non-Null Count   Dtype         
--------------   -----         
196726 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 1.5 MB


In [9]:
pbp_home_total = pbp_home[pbp_home['posteam_type'] == 'home']
pbp_home_total_sum = pbp_home_total.groupby(["old_game_id"]).agg({'home_team':'max',
                                                                  'away_team':'max',
                                                                  'shotgun':'sum',
                                                                  'total_home_score':'max',
                                                                  'air_yards':'mean',
                                                                  'yards_gained':'mean',
                                                                  'game_date':'max'})

In [10]:
# Calculate the rolling average (3-game window)
# Sort by date (ascending order)
pbp_home_total_sum.sort_values(by=['home_team', 'game_date'], inplace=True)

# Calculate the rolling average (3-game window) per team
pbp_home_total_sum['AirYards_Avg'] = pbp_home_total_sum['air_yards'].shift(1).rolling(window=2).mean()
pbp_home_total_sum['shotgun_Avg'] = pbp_home_total_sum['air_yards'].shift(1).rolling(window=2).mean()
pbp_home_total_sum['yards_gained_Avg'] = pbp_home_total_sum['air_yards'].shift(1).rolling(window=2).mean()
pbp_home_total_sum['home_total_Avg'] = pbp_home_total_sum['total_home_score'].shift(1).rolling(window=2).mean()

pbp_home_total_sum.tail()

,home_team,away_team,shotgun,total_home_score,air_yards,yards_gained,game_date,AirYards_Avg,shotgun_Avg,yards_gained_Avg,home_total_Avg
old_game_id,,,,,,,,,,,
2023102908,WAS,PHI,60.0,31.0,7.461538,5.363636,2023-10-29,7.318796,7.318796,7.318796,11.5
2023111907,WAS,NYG,64.0,19.0,6.600000,4.297872,2023-11-19,7.515083,7.515083,7.515083,25.5
2023120306,WAS,MIA,36.0,15.0,8.608696,3.528571,2023-12-03,7.030769,7.030769,7.030769,25.0
2023123110,WAS,SF,38.0,10.0,6.142857,3.813559,2023-12-31,7.604348,7.604348,7.604348,17.0
2024010707,WAS,DAL,36.0,10.0,5.500000,2.903226,2024-01-07,7.375776,7.375776,7.375776,12.5


In [11]:
### Preprocessing

#dropping missing vals
pbp_home_total_sum = pbp_home_total_sum.dropna()

In [12]:
#adding a year column based on the game_date column
pbp_home_total_sum['year'] = pbp_home_total_sum['game_date'].dt.year

In [13]:
# creating dummy variables from the home_team and year columns
pbp_home_total_sum_dummies = pd.get_dummies(pbp_home_total_sum, columns=['home_team','away_team'])

pbp_home_total_sum_dummies.head()

,shotgun,total_home_score,air_yards,yards_gained,game_date,AirYards_Avg,shotgun_Avg,yards_gained_Avg,home_total_Avg,year,...,away_team_NO,away_team_NYG,away_team_NYJ,away_team_PHI,away_team_PIT,away_team_SEA,away_team_SF,away_team_TB,away_team_TEN,away_team_WAS
old_game_id,,,,,,,,,,,,,,,,,,,,,
2020102508,78.0,37.0,8.062500,4.990385,2020-10-25,9.374436,9.374436,9.374436,26.5,2020,...,False,False,False,False,False,True,False,False,False,False
2020110809,66.0,31.0,9.615385,5.390244,2020-11-08,8.774107,8.774107,8.774107,30.0,2020,...,False,False,False,False,False,False,False,False,False,False
2020111506,71.0,32.0,7.218750,4.923913,2020-11-15,8.838942,8.838942,8.838942,34.0,2020,...,False,False,False,False,False,False,False,False,False,False
2020120608,62.0,28.0,6.153846,2.829268,2020-12-06,8.417067,8.417067,8.417067,31.5,2020,...,False,False,False,False,False,False,False,False,False,False
2020122011,67.0,33.0,9.837838,5.844444,2020-12-20,6.686298,6.686298,6.686298,30.0,2020,...,False,False,False,True,False,False,False,False,False,False


In [14]:
### AUSTIN CHANGED
#X = pbp_home_total_sum.loc[:, 'AirYards_Avg':'year_2024']
X = pbp_home_total_sum_dummies.drop(columns=['shotgun','total_home_score','air_yards','yards_gained','game_date'])
# this is also predicting with away team
y = pbp_home_total_sum['total_home_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Initialize the model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

model.score(X_test, y_test)

summary_df = pd.DataFrame({
    'Variable': X_train.columns,
    'Coefficient': model.coef_
})
summary_df

,Variable,Coefficient
0,AirYards_Avg,-1.082561e+14
1,shotgun_Avg,6.159441e+13
2,yards_gained_Avg,4.666165e+13
3,home_total_Avg,1.057129e-01
4,year,-4.526367e-01
...,...,...
64,away_team_SEA,-2.047700e+11
65,away_team_SF,-2.047700e+11
66,away_team_TB,-2.047700e+11
67,away_team_TEN,-2.047700e+11


In [17]:
import statsmodels.api as sm

start_col = 'home_team_ARI'
end_col = 'away_team_WAS'
columns_to_include = pbp_home_total_sum_dummies.columns[
    pbp_home_total_sum_dummies.columns.get_loc(start_col) + 1:
    pbp_home_total_sum_dummies.columns.get_loc(end_col)
]
pbp_home_total_dummy = pd.get_dummies(pbp_home_total_sum_dummies, columns=columns_to_include, drop_first=True)
pbp_home_total_dummy.replace({False: 0, True: 1}, inplace=True)

In [18]:
pbp_home_total_dummy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 2020102508 to 2024010707
Data columns (total 74 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   shotgun             1121 non-null   float64       
 1   total_home_score    1121 non-null   float64       
 2   air_yards           1121 non-null   float64       
 3   yards_gained        1121 non-null   float64       
 4   game_date           1121 non-null   datetime64[ns]
 5   AirYards_Avg        1121 non-null   float64       
 6   shotgun_Avg         1121 non-null   float64       
 7   yards_gained_Avg    1121 non-null   float64       
 8   home_total_Avg      1121 non-null   float64       
 9   year                1121 non-null   int32         
 10  home_team_ARI       1121 non-null   int64         
 11  away_team_WAS       1121 non-null   int64         
 12  home_team_ATL_True  1121 non-null   int64         
 13  home_team_BAL_True  1121 non-null   in

In [19]:
X = pbp_home_total_dummy.loc[:, 'AirYards_Avg':'away_team_TEN_True']
y = pbp_home_total_sum_dummies['total_home_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = sm.OLS(y_train, sm.add_constant(X_train)).fit()
summary_table = model.summary()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:       total_home_score   R-squared:                       0.159
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     2.409
Date:                Tue, 27 Aug 2024   Prob (F-statistic):           1.64e-08
Time:                        15:46:56   Log-Likelihood:                -3249.0
No. Observations:                 896   AIC:                             6630.
Df Residuals:                     830   BIC:                             6947.
Df Model:                          65                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                875.4468    531

In [21]:
pbp_home_total_sum_dummies.to_csv('pbp_home_final')